In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [2]:
data = pd.read_feather('database/GS.feather')

In [3]:
def rfm_variables(df):
    f_sales = len(df)
    f_profit = len(df[df['Profit'] > 0])
    m_sales = round(df['Sales'].sum(),2)
    m_profit = round(df['Profit'].sum(),2)
    m_qty = df['Quantity'].sum()
    m_deliver = round(df['Shipping Cost'].sum(),2)
    df_sort = df[['Order Date']].sort_values(by='Order Date').drop_duplicates()
    df_sort['diff'] = df_sort['Order Date'] - df_sort['Order Date'].shift(1)
    df_sort['diff_int'] = df_sort['diff'].dt.days
    r_days = round(df_sort['diff_int'].mean(),2)
    return f_sales, f_profit, m_sales, m_profit, m_qty, m_deliver, r_days

In [4]:
def fit_data(data, variable):
    rfm = pd.DataFrame()
    variaveis = data[variable].unique()
    for variavel in variaveis:
        dados = data[data[variable] == variavel]
        f_vendas, f_lucro, m_vendas, m_lucro, m_qtde, m_entrega, r_dias = rfm_variables(dados)
        new_row = {
            'referencia': variavel,
            'm_vendas': m_vendas,
            'm_lucro': m_lucro,
            'm_qtde': m_qtde,
            'm_entrega': m_entrega,
            'r_dias': r_dias,
            'f_vendas': f_vendas,
            'f_lucro': f_lucro
        }
        rfm = rfm.append(
            new_row,
            ignore_index = True
        )
    return rfm

In [5]:
original = fit_data(data, 'Country')
original = original.fillna(0)
original

/tmp/ipykernel_2212/952877166.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
/tmp/ipykernel_2212/952877166.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
/tmp/ipykernel_2212/952877166.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
/tmp/ipykernel_2212/952877166.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
/tmp/ipykernel_2212/952877166.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
/tmp/ipykernel_2212/952877166.py:17: FutureWarning

In [6]:
variaveis = ['m_vendas', 'm_lucro', 'm_qtde', 'm_entrega', 'r_dias', 'f_vendas', 'f_lucro']

In [7]:
base = original[variaveis]
base

,m_vendas,m_lucro,m_qtde,m_entrega,r_dias,f_vendas,f_lucro
0,622590.62,102818.10,10011,67658.40,1.89,2644,2272
1,81502.53,18798.05,1277,9411.77,9.32,360,350
2,361106.42,30090.50,6148,38170.04,2.66,1599,1185
3,149687.06,33401.44,2235,17155.17,5.30,614,600
4,51539.93,-17723.45,1426,5416.04,7.78,388,75
...,...,...,...,...,...,...,...
142,850.62,290.16,18,81.54,358.67,7,7
143,150.51,44.46,3,9.38,0.00,3,3
144,669.18,12.84,6,72.76,0.00,2,2
145,1317.03,90.00,11,297.94,0.00,2,1


In [ ]:
vizinhos = NearestNeighbors(n_neighbors=min(4, len(base))).fit(base)
similares = pd.DataFrame()
for index, row in original.iterrows():
    print('Referencia: {0}'.format(row['referencia']))
    print('Referencias Similares:')
    original_referencia = original[original['referencia'] == row['referencia']][variaveis]
    similar = vizinhos.kneighbors(original_referencia, return_distance=False)[0]
    original_similar = original.iloc[similar][variaveis].reset_index()
    referencia = original.iloc[similar]['referencia'].reset_index()
    referencia = referencia.merge(original_similar, on='index', how='left')
    referencia = referencia.drop(columns=['index'])
    for ind, rw in referencia.iterrows():    
        if row['referencia'] != rw['referencia']:            
            print('--> {0}'.format(rw['referencia']))
            similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
similares

In [9]:
similares.to_feather('database/knn_produto.feather')

In [ ]:
## Aula 15 - KNN - Vizinhos mais próximos
original = fit_data(data, 'Sub-Category')
original = original.fillna(0)
base = original[variaveis]
vizinhos = NearestNeighbors(n_neighbors=min(4, len(base))).fit(base)
similares = pd.DataFrame()
for index, row in original.iterrows():
    print('Referencia: {0}'.format(row['referencia']))
    print('Referencias Similares:')
    original_referencia = original[original['referencia'] == row['referencia']][variaveis]
    similar = vizinhos.kneighbors(original_referencia, return_distance=False)[0]
    original_similar = original.iloc[similar][variaveis].reset_index()
    referencia = original.iloc[similar]['referencia'].reset_index()
    referencia = referencia.merge(original_similar, on='index', how='left')
    referencia = referencia.drop(columns=['index'])
    for ind, rw in referencia.iterrows():    
        if row['referencia'] != rw['referencia']:            
            print('--> {0}'.format(rw['referencia']))
            similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
similares

similares.to_feather('database/knn_subcategoria.feather')

In [ ]:
## Aula 15 - KNN - Vizinhos mais próximos
original = fit_data(data, 'Product Name')
original = original.fillna(0)
base = original[variaveis]
vizinhos = NearestNeighbors(n_neighbors=min(4, len(base))).fit(base)
similares = pd.DataFrame()
for index, row in original.iterrows():
    print('Referencia: {0}'.format(row['referencia']))
    print('Referencias Similares:')
    original_referencia = original[original['referencia'] == row['referencia']][variaveis]
    similar = vizinhos.kneighbors(original_referencia, return_distance=False)[0]
    original_similar = original.iloc[similar][variaveis].reset_index()
    referencia = original.iloc[similar]['referencia'].reset_index()
    referencia = referencia.merge(original_similar, on='index', how='left')
    referencia = referencia.drop(columns=['index'])
    for ind, rw in referencia.iterrows():    
        if row['referencia'] != rw['referencia']:            
            print('--> {0}'.format(rw['referencia']))
            similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
similares

similares.to_feather('database/knn_produto.feather')

In [ ]:
original = fit_data(data, 'Country')
original = original.fillna(0)
base = original[variaveis]
vizinhos = NearestNeighbors(n_neighbors=min(4, len(base))).fit(base)
similares = pd.DataFrame()
for index, row in original.iterrows():
    print('Referencia: {0}'.format(row['referencia']))
    print('Referencias Similares:')
    original_referencia = original[original['referencia'] == row['referencia']][variaveis]
    similar = vizinhos.kneighbors(original_referencia, return_distance=False)[0]
    original_similar = original.iloc[similar][variaveis].reset_index()
    referencia = original.iloc[similar]['referencia'].reset_index()
    referencia = referencia.merge(original_similar, on='index', how='left')
    referencia = referencia.drop(columns=['index'])
    for ind, rw in referencia.iterrows():    
        if row['referencia'] != rw['referencia']:            
            print('--> {0}'.format(rw['referencia']))
            similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
similares

similares.to_feather('database/knn_pais.feather')